In [1]:
from emojify.nn_manager import NNManager
from emojify.lstm_bert_embeddings import ExampleNet
import torch
import emojify.utils as utils
import seaborn as sns
import matplotlib.pyplot as plt
from emojify.transformer_data import load_transformer_data

c:\Users\ugioh\code\emojify\venv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:


embedding_dim = 384  # should probably be 32
hidden_dim = 32  # should probably be 32
epochs = 30
batch_size = 32
learning_rate = 5e-3
early_stop_count = 4
train, val, test = load_transformer_data(batch_size)

classes [4, 4, 0, 3, 0, 4, 5, 1, 2, 3, 4, 2, 0, 4, 2, 2, 4, 4, 4, 1, 0, 1, 2, 2, 0, 4, 4, 4, 0, 2, 2, 1, 5, 0, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 4, 4, 2, 3, 2, 0, 2, 4, 0, 1, 2, 4, 4, 5, 2, 2, 2, 3, 1, 1, 5, 0, 0, 4, 3, 2, 4, 4, 2, 4, 4, 4, 2, 2, 2, 0, 4, 0, 0, 0, 2, 2, 2, 2, 4, 1, 3, 0, 4, 0, 3, 4, 2, 2, 4, 0, 3, 2, 2, 2, 4, 2, 2, 2, 2, 4, 2, 2, 3, 4, 4, 2, 2, 4, 2, 2, 1, 1, 1, 4, 3, 2, 2, 3, 1, 5, 2, 2, 2, 2, 0, 1, 2, 0, 3, 0, 4, 2, 4, 0, 2, 5, 4, 0, 0, 4, 2, 1, 2, 2, 1, 4, 5, 5, 2, 0, 1, 0, 4, 0, 4, 1, 4, 2, 5, 1, 2, 0, 2, 0, 2, 1, 1, 4, 2, 2, 4, 5, 2, 1, 2, 4, 2, 2, 2, 1, 2, 5, 2, 0, 4, 4, 2, 0, 4, 2, 2, 5, 2, 1, 2, 1, 4, 1, 3, 4, 1, 3, 4, 4, 0, 2, 3, 0, 4, 5, 0, 2, 4, 0, 2, 2, 2, 2, 2, 4, 2, 3, 2, 4, 0, 5, 2, 0, 2, 3, 4, 4, 2, 4, 4, 2, 4, 2, 2, 2, 1, 4, 0, 4, 0, 3, 2, 3, 4, 2, 2, 2, 4, 2, 4, 0, 3, 4, 3, 4, 4, 4, 4, 4, 4, 1, 2, 4, 4, 2, 3, 4, 0, 4, 3, 2, 2, 2, 4, 2, 4, 2, 2, 2, 4, 4, 4, 3, 1, 4, 1, 2, 2, 1, 2, 0, 0, 3, 4, 5, 2, 0, 4, 4, 0, 3, 5, 4, 3, 2, 0, 2, 4, 4, 2, 2, 1, 4, 0, 2, 3

In [3]:
model = ExampleNet(embedding_dim, hidden_dim, batch_size)
nn_manager = NNManager(
    batch_size,
    learning_rate,
    early_stop_count,
    epochs,
    model,
    (train, val, test),
    modify_model_output=lambda out: out[0],
)
nn_manager.train()

ExampleNet(
  (lstm): LSTM(384, 32, batch_first=True)
  (dropout): Dropout(p=0.2, inplace=False)
  (fc): Linear(in_features=32, out_features=3, bias=True)
)
---- Epoch 1 of 30 ----


  0%|          | 0/500 [00:00<?, ?it/s]


TypeError: ExampleNet.forward() missing 1 required positional argument: 'hidden'

In [ ]:
accuracy = nn_manager.test()
accuracy

100%|██████████| 63/63 [00:00<00:00, 1032.81it/s]

Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       265
           1       0.00      0.00      0.00       242
           2       0.00      0.00      0.00       677
           3       0.00      0.00      0.00       144
           4       0.30      1.00      0.46       596
           5       0.00      0.00      0.00        76

    accuracy                           0.30      2000
   macro avg       0.05      0.17      0.08      2000
weighted avg       0.09      0.30      0.14      2000




c:\Users\ugioh\code\emojify\venv\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\ugioh\code\emojify\venv\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\ugioh\code\emojify\venv\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


0.298

In [ ]:
class_to_idx = {'anger': 0, 'fear': 1, 'joy': 2, 'love': 3, 'sadness': 4, 'surprise': 5}
idx_to_class = {v: k for k, v in class_to_idx.items()}
from emojify.transformer_embeddings import get_transformer_embeddings
import emojify.utils as utils

def get_emoji(sentence):
    model.eval()
    with torch.no_grad():
        sentence = get_transformer_embeddings(sentence)
        input_ = utils.to_cuda(torch.tensor(sentence).reshape(1, -1))
        tag_scores = model(input_)
        predicted = tag_scores.argmax(dim=1)
        return idx_to_class[predicted.item()]


In [ ]:
get_emoji("This is a very beautiful day")


'sadness'

In [ ]:
get_emoji("I hate you")

'sadness'

In [ ]:
get_emoji("What the hell is wrong with you")

'sadness'